# **PROYECTO**

## COSMIC DEFENDER

In [25]:
import pygame #Se importa la librería pygame con todas sus funciones.
import random #Se importa la librería random para generación de eventos aleatorios.
import time

In [26]:
pygame.init() #Se inicia el constructor de pygame
pygame.mixer.init() #Se inicia el constructor de sonido de pygame.

## Configuraciones pantalla de inicio

In [27]:
anchoPantalla=800  #Se crea una variable con el ancho de pantalla.
altoPantalla=600   #Se crea una variable con el alto de pantalla.

#Se crea la pantalla del juego.
pantalla=pygame.display.set_mode((anchoPantalla,altoPantalla))
pygame.display.set_caption("Cosmic Defenders")

#Se ingresa la imagen de inicio en la pantalla.
pantallaDeInicio=pygame.image.load("Inicio-PrimerNivel.png").convert()
pantallaDeInicio=pygame.transform.scale(pantallaDeInicio,(800, 600))

#Se ingresa el nombre del juego para mostrar en pantalla.
fuenteTitulo=pygame.font.Font("Silkscreen-Bold.ttf",50)
textoTitulo=fuenteTitulo.render("Cosmic Defenders",True, (0, 0, 0))
textoRect1=textoTitulo.get_rect(center=(anchoPantalla // 2, altoPantalla // 6))

#Se crea el botón de inicio del juego.
fuenteBotones=pygame.font.Font("Silkscreen-Bold.ttf", 40)
textoInicio=fuenteBotones.render("INICIO", True, (255, 255, 255))
textoRect2=textoInicio.get_rect(center=(anchoPantalla // 2, altoPantalla // 1.5))
inicio=pygame.Rect(textoRect2.left - 1, textoRect2.top - 1, textoRect2.width + 1, textoRect2.height + 1)

#Se crea el botón de ajustes del juego.
textoAjustes=fuenteBotones.render("AJUSTES", True, (255, 255, 255))
textoRect3=textoAjustes.get_rect(center=(anchoPantalla // 2, altoPantalla // 1.3))
ajustes=pygame.Rect(textoRect3.left - 1, textoRect3.top - 1, textoRect3.width + 1, textoRect3.height + 1)

#Se crea la variable con el sonido de fondo.
musicaInicio=pygame.mixer.music.load("Himno Universidad de Santiago De Chile.mp3")
reproducirMusica=pygame.mixer.music.play(-1)

#Se crea la variable que contiene el sonido del disparo de la nave
sonidoDisparo = pygame.mixer.Sound("piun.wav")

# Diccionario de música por nivel
musica_niveles = {
    1: "primer_nivel.mp3",
    2: "segundo_nivel.mp3",
    3: "tercer_nivel.mp3"
}

## Clases

In [28]:
# Se crea la clase Jugador
class Jugador(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.imagen = pygame.image.load("Nave.png").convert_alpha()  # Cargar la imagen de la nave
        self.imagen = pygame.transform.scale(self.imagen, (50, 50))
        self.rect = self.imagen.get_rect()
        self.rect.center = (anchoPantalla // 2, altoPantalla - 50)
        self.velocidad = 10
        self.proyectiles = pygame.sprite.Group()
        self.enemigos_derribados = 0

    def mover(self, teclas):
        if teclas[pygame.K_LEFT] and self.rect.left > 0:
            self.rect.x -= self.velocidad
        if teclas[pygame.K_RIGHT] and self.rect.right < anchoPantalla:
            self.rect.x += self.velocidad

    def aparicion(self, pantalla):
        pantalla.blit(self.imagen, self.rect)

    def disparar(self):
        proyectil = Proyectil(self.rect.centerx, self.rect.top)
        self.proyectiles.add(proyectil)

    def actualizar(self, meteoritos, puntaje):
        for proyectil in self.proyectiles:
            proyectil.mover()  # Mover el proyectil
            proyectil.aparecer(pantalla)  # Mostrar el proyectil
            # Verificar si el proyectil colide con algún meteorito
            for meteorito in meteoritos:
                if proyectil.rect.colliderect(meteorito.rect):  # Si colisiona
                    puntos_obtenidos = meteorito.recibir_disparo()  # Obtener el puntaje del meteorito
                    puntaje += puntos_obtenidos  # Sumar los puntos al puntaje
                    proyectil.kill()  # El proyectil es destruido
                    self.enemigos_derribados += 1
        return puntaje

# Se crea la clase Proyectil
class Proyectil(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.imagen=pygame.image.load("proyectil.png").convert_alpha()  # Cargar imagen del proyectil
        self.imagen=pygame.transform.scale(self.imagen, (10, 30))  # Tamaño del proyectil
        self.rect=self.imagen.get_rect()  # Obtenemos el rectángulo del proyectil
        self.rect.centerx=x  # Posición horizontal inicial
        self.rect.bottom=y  # Posición vertical inicial
        self.velocidad=7  # Velocidad del proyectil

    def mover(self):
        self.rect.y-=self.velocidad  # se mueve el proyectil hacia arriba

        # Si el proyectil ha salido de la pantalla, no se actualiza
        if self.rect.bottom < 0:
            self.kill()  # El proyectil es eliminado del grupo de sprites

    def aparecer(self, pantalla):
        pantalla.blit(self.imagen, self.rect)  # Se muestra el proyectil en pantalla

class Enemigos(pygame.sprite.Sprite):
    def __init__(self, enemigos, velocidad, tamano, disparos_necesarios,puntaje):
        super().__init__()
        self.imagen = pygame.image.load(enemigos).convert_alpha()  # Cargar imagen del meteorito
        self.imagen = pygame.transform.scale(self.imagen, tamano)  # Tamaño del meteorito
        self.rect = self.imagen.get_rect()  # Obtenemos el rectángulo del meteorito
        self.velocidad = velocidad  # Velocidad del meteorito
        self.disparos_recibidos = 0  # Contador de disparos recibidos
        self.disparos_necesarios = disparos_necesarios  # Número de disparos necesarios para 
        self.puntaje=puntaje

    def caer(self):
        self.rect.y += self.velocidad  # El meteorito se mueve hacia abajo

    def mostrar(self, pantalla):
        pantalla.blit(self.imagen, self.rect)  # Mostrar meteorito en pantalla

    def recibir_disparo(self):
        # Incrementa el contador de disparos recibidos
        self.disparos_recibidos += 1
        # Si se han recibido suficientes disparos, destruye el meteorito
        if self.disparos_recibidos >= self.disparos_necesarios:
            self.kill()  # El meteorito es destruido
            return self.puntaje
        return 0


class meteorito1(Enemigos):
    def __init__(self):
        super().__init__("meteorito1.png", velocidad=1, tamano=(50, 70), disparos_necesarios=1,puntaje=10)
        x = random.randint(0, anchoPantalla - 70)
        y = 0
        self.rect = pygame.Rect(x, y, 50, 70)

class meteorito2(Enemigos):
    def __init__(self):
        super().__init__("meteorito2.png", velocidad=2, tamano=(50, 70), disparos_necesarios=2,puntaje=20)
        x = random.randint(0, anchoPantalla - 70)
        y = 0
        self.rect = pygame.Rect(x, y, 50, 70)

class meteorito3(Enemigos):
    def __init__(self):
        super().__init__("meteorito3.png", velocidad=1, tamano=(120, 150), disparos_necesarios=4,puntaje=40)
        x = random.randint(0, anchoPantalla - 150)
        y = 0
        self.rect = pygame.Rect(x, y, 120, 150)
        

## Funciones

In [29]:
  
def pantallaAjustes():
    """
    Funcion para definir la pantalla de ajustes

    Retorna:
    --------------------
    textoRectSalir = Variable para salir de la pantalla de ajustes
    textoRectOnMusica = Variable para encender la musica 
    textoRectOffMusica = Variable para apagar la musica
    textoRectOnSonido = Variable para encender el sonido del juego
    textoRectOffSonido = Variable para apgar el sonido del juego
    """
    pantalla.fill((0, 0, 0))  #Llenamos la pantalla de negro

    #Se crea el botón de ajustar musica del juego.
    fuenteAjustes=pygame.font.Font("Silkscreen-Bold.ttf", 50)
    textoMusica=fuenteAjustes.render("Musica", True, (255, 255, 255))
    textoRectMusica=textoMusica.get_rect(center=(anchoPantalla // 2, altoPantalla // 5))

    #Se crea el botón de encender la música.
    fuenteOnMusica=pygame.font.Font("Silkscreen-Bold.ttf", 30)
    textoOnMusica=fuenteOnMusica.render("ON", True, (255, 255, 255))
    textoRectOnMusica=textoOnMusica.get_rect(center=(anchoPantalla // 2.3, altoPantalla // 3))
    pygame.draw.rect(pantalla, (0, 255, 0), textoRectOnMusica)

    #Se crea el botón de apagar la música.
    fuenteOffMusica=pygame.font.Font("Silkscreen-Bold.ttf", 30)
    textoOffMusica=fuenteOffMusica.render("OFF", True, (255, 255, 255))
    textoRectOffMusica=textoOffMusica.get_rect(center=(anchoPantalla // 1.7, altoPantalla // 3))
    pygame.draw.rect(pantalla, (255, 0, 0), textoRectOffMusica)

    #Se crea el botón de ajustar sonido del juego.
    fuenteSonido=pygame.font.Font("Silkscreen-Bold.ttf", 50)
    textoSonido=fuenteSonido.render("Sonidos", True, (255, 255, 255))
    textoRectSonido=textoSonido.get_rect(center=(anchoPantalla // 2, altoPantalla // 2))

    #Se crea el botón de encender el sonido.
    fuenteOnSonido=pygame.font.Font("Silkscreen-Bold.ttf", 30)
    textoOnSonido=fuenteOnSonido.render("ON", True, (255, 255, 255))
    textoRectOnSonido=textoOnSonido.get_rect(center=(anchoPantalla // 2.3, altoPantalla // 1.6))
    pygame.draw.rect(pantalla, (0, 255, 0), textoRectOnSonido)

    #Se crea el botón de apagar el sonido.
    fuenteOffSonido=pygame.font.Font("Silkscreen-Bold.ttf", 30)
    textoOffSonido=fuenteOffSonido.render("OFF", True, (255, 255, 255))
    textoRectOffSonido=textoOffSonido.get_rect(center=(anchoPantalla // 1.7, altoPantalla // 1.6))
    pygame.draw.rect(pantalla, (255, 0, 0), textoRectOffSonido)

    #Se crea el botón de salir de los ajustes.
    fuenteSalir=pygame.font.Font("Silkscreen-Bold.ttf", 50)
    textoSalir=fuenteSalir.render("Salir", True, (255, 255, 255))
    textoRectSalir=textoSalir.get_rect(center=(anchoPantalla // 2, altoPantalla // 1.3))

    #Se muestran los botones en pantalla.
    pantalla.blit(textoMusica,textoRectMusica)
    pantalla.blit(textoOnMusica,textoRectOnMusica)
    pantalla.blit(textoOffMusica,textoRectOffMusica)
    pantalla.blit(textoSonido,textoRectSonido)
    pantalla.blit(textoOnSonido,textoRectOnSonido)
    pantalla.blit(textoOffSonido,textoRectOffSonido)
    pantalla.blit(textoSalir,textoRectSalir)

    return textoRectSalir,textoRectOnMusica,textoRectOffMusica,textoRectOnSonido,textoRectOffSonido

def pantallaPrimerNivel():
    """
    Funcion para mostrar la pantalla del primer nivel
    """
    #Se ingresa la imagen de inicio en la pantalla.
    pantallaPrimerNivel=pygame.image.load("Inicio-PrimerNivel.png").convert()
    pantallaPrimerNivel=pygame.transform.scale(pantallaPrimerNivel, (800, 600))

    #Se muestra la pantalla del primer nivel.
    pantalla.blit(pantallaPrimerNivel,(0,0))
    
# Funcion para mostrar puntaje, luego actualizar para mostrar puntajes nuevos
def mostrarPuntaje(pantalla, puntaje):
    """
    Funcion para mostrar el puntaje en pantalla

    Parametros:
    ----------------
    pantalla = Varible que representa la pantalla del video 
    puntaje = Variable que representa el puntaje actual del jugador
    """
    fuentePuntaje=pygame.font.Font("Silkscreen-Bold.ttf", 22)
    textoPuntaje=fuentePuntaje.render(f"Puntuación: {puntaje}", True, (255, 255, 255))
    pantalla.blit(textoPuntaje,(anchoPantalla-300,10))
    
def reproducir_musica(nivel):
    """
    Función para cargar y reproducir música de nivel
    """
    if nivel in musica_niveles:
        pygame.mixer.music.stop()  # Detener música actual
        pygame.mixer.music.load(musica_niveles[nivel])  # Cargar nueva música
        pygame.mixer.music.play(-1)  # Reproducir en bucle

# Variables de control del juego
niveles=1  
meteoritos=[]  
crearMeteoritos=0  
tiempoDeCreacion=5500  
cantidadDeMeteoritos=100  
contadorDeMeteoritos=0  

# Función para generar meteoritos
def enemigosPorNivel(niveles, meteoritos, crearMeteoritos, tiempoDeCreacion, cantidadDeMeteoritos, contadorDeMeteoritos, jugador, nivel_cambiado):
    """
    Funcion para general meteoritos

    Parametros:
    ------------
    niveles = Nivel actual
    meteoritos = Lista para almacenar los meteoritos
    crearMeteoritos = Controlador del tiempo para crear meteoritos
    tiempoDeCreacion = Intervalo en milisegundos para crear meteoritos
    cantidadDeMeteoritos = Número total de meteoritos por nivel
    contadorDeMeteoritos = Contador para saber cuántos meteoritos han sido generados
    jugador = Instancia de jugador

    Retorna:
    -----------
    meteoritos (list) = lista de meteoritos 
    crearMeteoritos (Int) =  
    contadorDeMeteoritos =
    """
    # Controlamos el tiempo entre meteoritos
    tiempoNivel = pygame.time.get_ticks()  # Tiempo desde el inicio
    if (tiempoNivel - crearMeteoritos >= tiempoDeCreacion) and (contadorDeMeteoritos < cantidadDeMeteoritos):
        if niveles == 1:
            filaDeMeteoritos = 5
            for i in range(filaDeMeteoritos):
                meteoritos.add(meteorito1())  # Meteoritos de tipo 1 (1 disparo necesario)
        
            if jugador.enemigos_derribados >= 10:  # 10 enemigos para pasar al nivel 2
                niveles += 1  # Avanzamos al nivel 2
                jugador.enemigos_derribados = 0
                nivel_cambiado = True # Se activa el siguiente nivel
                meteoritos.empty()


        # A continuación se pueden añadir más niveles con diferentes tipos de meteoritos.
        elif niveles == 2:
            filaDeMeteoritos = 6
            for i in range(filaDeMeteoritos):
                meteoritos.add(meteorito1())  

            if jugador.enemigos_derribados >= 10: 
                niveles += 1  
                jugador.enemigos_derribados = 0
                nivel_cambiado = True
                meteoritos.empty()
        
        elif niveles == 3:
            filaDeMeteoritos = 4
            for i in range(filaDeMeteoritos):
                meteoritos.add(meteorito1())  

            if jugador.enemigos_derribados >= 10:  
                niveles += 1 
                jugador.enemigos_derribados = 0
                nivel_cambiado = True
                meteoritos.empty()

        elif niveles == 4:
            filaDeMeteoritos = 4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                if numeroRandom<30:
                    meteoritos.add(meteorito2())  
                else:
                    meteoritos.add(meteorito1())
            if jugador.enemigos_derribados >= 10:  
                niveles += 1  
                jugador.enemigos_derribados = 0
                nivel_cambiado = True
                meteoritos.empty()

        elif niveles == 5:
            filaDeMeteoritos = 4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                if numeroRandom<50:
                    meteoritos.add(meteorito2())  
                else:
                    meteoritos.add(meteorito1())
            if jugador.enemigos_derribados >= 10:  
                niveles += 1 
                jugador.enemigos_derribados = 0
                nivel_cambiado = True
                meteoritos.empty()

        elif niveles == 6:
            filaDeMeteoritos = 4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                if numeroRandom<60:
                    meteoritos.add(meteorito2())  
                else:
                    meteoritos.add(meteorito1())
            if jugador.enemigos_derribados >= 10:  
                niveles += 1  
                jugador.enemigos_derribados = 0
                nivel_cambiado = True
                meteoritos.empty()

        elif niveles == 7:
            filaDeMeteoritos = 4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                numeroRandom2=random.randrange(0,100,1)

                if numeroRandom<60:
                    meteoritos.add(meteorito2())  
                if numeroRandom2<30:
                    meteoritos.add(meteorito3())
                else:
                    meteoritos.add(meteorito1())
            if jugador.enemigos_derribados >= 10:  
                niveles += 1  
                jugador.enemigos_derribados = 0
                nivel_cambiado = True
                meteoritos.empty()

        elif niveles == 8:
            filaDeMeteoritos = 4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                numeroRandom2=random.randrange(0,100,1)

                if numeroRandom<60:
                    meteoritos.add(meteorito2())  
                if numeroRandom2<40:
                    meteoritos.add(meteorito3())
                else:
                    meteoritos.add(meteorito1())
            if jugador.enemigos_derribados >= 10:  
                niveles += 1  
                jugador.enemigos_derribados = 0
                nivel_cambiado = True
                meteoritos.empty()

        elif niveles == 9:
            filaDeMeteoritos = 4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                numeroRandom2=random.randrange(0,100,1)

                if numeroRandom<60:
                    meteoritos.add(meteorito2())  
                if numeroRandom2<50:
                    meteoritos.add(meteorito3())
                else:
                    meteoritos.add(meteorito1())
            if jugador.enemigos_derribados >= 10:  
                niveles += 1  
                jugador.enemigos_derribados = 0
                nivel_cambiado = True


        contadorDeMeteoritos += filaDeMeteoritos  # Incrementamos el contador de meteoritos
        crearMeteoritos = tiempoNivel  # Actualizamos el tiempo del último meteorito creado

    return meteoritos, crearMeteoritos, contadorDeMeteoritos, niveles, nivel_cambiado

def mostrarMensaje(pantalla, mensaje, color, pos_x, pos_y, font_size=36):
    """ Función para mostrar un mensaje en la pantalla """
    fuente = pygame.font.Font(None, font_size)
    texto = fuente.render(mensaje, True, color)
    rect = texto.get_rect(center=(pos_x, pos_y))
    pantalla.blit(texto, rect)
    pygame.display.flip()

## Ejecucion

In [30]:
import pygame
import time

pygame.init()
pygame.mixer.init()  # Inicializar el mixer para reproducir sonidos

def reproducir_musica(nivel):
    """Reproduce la música correspondiente al nivel."""
    try:
        pygame.mixer.music.stop()  # Detener cualquier música en reproducción
        if nivel == 1:
            archivo_musica = "primer_nivel.mp3"
        elif nivel == 2:
            archivo_musica = "segundo_nivel.mp3"
        elif nivel == 3:
            archivo_musica = "tercer_nivel.mp3"
        else:
            archivo_musica = "musica_default.mp3"  # Música por defecto

        print(f"Reproduciendo música del nivel {nivel}: {archivo_musica}")
        pygame.mixer.music.load(archivo_musica)
        pygame.mixer.music.play(-1)  # Reproducir en bucle
    except pygame.error as e:
        print(f"Error al cargar o reproducir música: {e}")

# Configuración inicial
clock = pygame.time.Clock()
nave = Jugador()
meteoritos = pygame.sprite.Group()
puntaje = 0
ultimoTiempo = time.time()

ejecucion = True
ingresarAjustes = False
inicioJuego = False
escucha = True
miusik = True
nivel_actual = 0
enemigos_derribados = 0
nivel_cambiado = False
tiempo_espera_nivel = 2000
ultimo_tiempo_nivel = time.time()

# Se hace un ciclo para ejecutar el juego.
while ejecucion:
    for opcionesJuego in pygame.event.get():
        if opcionesJuego.type == pygame.QUIT:  # Si se presiona la salida de la pestaña de juego, se termina la ejecución.
            ejecucion = False

        if opcionesJuego.type == pygame.MOUSEBUTTONDOWN:
            if ajustes.collidepoint(opcionesJuego.pos):  # Si se hace clic en el botón de ajustes.
                ingresarAjustes = True  # Cambiamos a pantalla de ajustes.

            if inicio.collidepoint(opcionesJuego.pos) and ingresarAjustes == False:  # Si se hace clic en el botón de inicio.
                inicioJuego = True  # Cambiamos a pantalla de primer nivel.
                nivel_actual = 1
                if miusik == True:
                    reproducir_musica(nivel_actual)  # Reproducir música del nivel
                else:
                    pygame.mixer.music.stop()

        if opcionesJuego.type == pygame.KEYDOWN:
            if opcionesJuego.key == pygame.K_SPACE:  # Si se presiona la tecla de disparo (espacio)
                nave.disparar()  # Dispara un proyectil desde la nave
                if escucha == True:
                    sonidoDisparo.play(loops=0)
                else:
                    sonidoDisparo.stop()

    if inicioJuego:
        pantallaPrimerNivel()  # Mostramos la pantalla del primer nivel.
        teclas = pygame.key.get_pressed()  # Obtenemos las teclas presionadas
        nave.mover(teclas)  # Movemos al jugador según las teclas presionadas
        nave.aparicion(pantalla)  # Mostramos la pantalla del primer nivel
        meteoritos, crearMeteoritos, contadorDeMeteoritos, nivel_actual, nivel_cambiado = enemigosPorNivel(
            nivel_actual, meteoritos, crearMeteoritos, tiempoDeCreacion, cantidadDeMeteoritos, contadorDeMeteoritos, nave, nivel_cambiado
        )

        for meteorito in meteoritos:
            meteorito.caer()
            meteorito.mostrar(pantalla)

        puntaje = nave.actualizar(meteoritos, puntaje)
        # Se configura la puntuación
        if time.time() - ultimoTiempo >= 1:  # Si ha pasado 1 segundo
            puntaje += 1  # Incrementamos la puntuación
            ultimoTiempo = time.time()  # Actualizamos el último tiempo de incremento de puntuación

        if nivel_cambiado == True:
            mostrarMensaje(pantalla, f"Nivel {nivel_actual - 1} superado", (255, 255, 255), 400, 300, font_size=48)
            pygame.display.update()
            pygame.time.delay(tiempo_espera_nivel)
            if miusik == True:
                reproducir_musica(nivel_actual)  # Cambiar música al nuevo nivel
            nivel_cambiado = False  # Marcar que el nivel ha cambiado

        mostrarPuntaje(pantalla, puntaje)  # Se muestra el puntaje en pantalla
        pygame.display.update()  # Se actualiza la pantalla

    if ingresarAjustes:
        pantallaAjustes()  # Mostramos la pantalla de ajustes.
        salirAjustes, onMusica, offMusica, onSonido, offSonido = pantallaAjustes()
        pygame.display.update()  # Se actualiza la pantalla.

        for opcionesAjustes in pygame.event.get():
            if opcionesAjustes.type == pygame.MOUSEBUTTONDOWN:
                if offMusica.collidepoint(opcionesAjustes.pos):
                    pygame.mixer.music.stop()
                    miusik = False

                if onMusica.collidepoint(opcionesAjustes.pos):
                    if pygame.mixer.music.get_busy() == False:
                        pygame.mixer.music.play(-1)
                        miusik = True

                if offSonido.collidepoint(opcionesAjustes.pos):
                    escucha = False  # Si se apreta el botón off no se escuchará

                if onSonido.collidepoint(opcionesAjustes.pos):
                    escucha = True  # Si se apreta el botón on se escuchará

            if opcionesAjustes.type == pygame.QUIT:  # Si se presiona la salida de la pestaña de juego, se termina la ejecución.
                ejecucion = False

            if opcionesAjustes.type == pygame.MOUSEBUTTONDOWN:
                if salirAjustes.collidepoint(opcionesAjustes.pos):  # Si se presiona la salida de los ajustes, se vuelve a la pantalla de inicio.
                    ingresarAjustes = False

    if (ingresarAjustes == False) and (inicioJuego == False):
        # Si no estamos en ajustes, mostramos la pantalla inicial.
        pantalla.blit(pantallaDeInicio, [0, 0])
        pantalla.blit(textoTitulo, textoRect1)

        pygame.draw.rect(pantalla, (160, 32, 240), inicio)  # Botón de INICIO.
        pantalla.blit(textoInicio, textoRect2)

        pygame.draw.rect(pantalla, (160, 32, 240), ajustes)  # Botón de AJUSTES.
        pantalla.blit(textoAjustes, textoRect3)

    pygame.display.flip()  # Se actualiza la pantalla.
    clock.tick(60)  # Se limita a 60 FPS

pygame.quit()



Reproduciendo música del nivel 1: primer_nivel.mp3
Reproduciendo música del nivel 2: segundo_nivel.mp3
